<a href="https://colab.research.google.com/github/shukurullo2004/Machine-learnings/blob/main/2_Copy_of_fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np


In [ ]:
train_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True,
    target_transform = None
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    transform = ToTensor(),
    download = True,
    target_transform=None
)


In [ ]:
len(train_data), len(test_data)

In [ ]:
image, label = train_data[0]

In [ ]:
classname = train_data.class_to_idx
classname

In [ ]:
classname = train_data.classes
classname

In [ ]:
classname[label]

In [ ]:
print(f"image shape: {image.shape}")
plt.imshow(image.squeeze())
plt.xlabel('weight')
plt.ylabel('height')
plt.title(label)

In [ ]:
plt.imshow(image.squeeze(), cmap = 'gray')
plt.title(classname[label])
plt.axis(False)

In [ ]:
# NOT WORKING THIS LINE ->torch.manual_seed(42)
row, col = 4,4
fig = plt.figure(figsize = (9,9))

for i in range(1,row*col+1):
  plt.title(classname[label])
  random_idx = torch.randint(0, len(train_data), size = [1]).item()
  img, label = train_data[random_idx]
  fig.add_subplot(row, col,i)
  plt.axis(False)
  plt.imshow(img.squeeze(), cmap = 'gray')


In [ ]:
from torch.utils.data import DataLoader
train_data_loader = DataLoader(dataset=train_data,
                               batch_size = 32,
                               shuffle = True)
test_data_loader = DataLoader(dataset=test_data,
                              batch_size= 32,
                              shuffle = False)
len(train_data_loader), len(test_data_loader)

In [ ]:
train_data,test_data

In [ ]:
from torch import nn

class FashionModule(nn.Module):
  def __init__ (self,
               in_shape : int,
               hidden_shape1: int,
               hidden_shape2: int,
               output_shape : int):

    super().__init__()
    self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=in_shape, out_features=hidden_shape1),
        nn.ReLU(),
        nn.Linear(hidden_shape1,hidden_shape2),
        nn.ReLU(),
        nn.Linear(hidden_shape2,hidden_shape2),
        nn.ReLU(),
        nn.Linear(hidden_shape2,output_shape),
    )

  def forward(self, x):
      return self.layer_stack(x)


In [ ]:
model_0 = FashionModule(in_shape=28*28,
                        hidden_shape1=256,
                        hidden_shape2 = 64,
                        output_shape= 10)
model_0

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(),lr=0.001)

import sklearn.metrics
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
train_data_loader.dataset

In [ ]:
import torch

def accuracy_fn(y_true, y_pred):
    _, predicted_classes = y_pred.max(dim=1)
    correct = (y_true == predicted_classes).sum().item()
    acc = (correct / len(y_true)) * 100
    return acc


In [ ]:
from tqdm.auto import tqdm
epochs = 3
train_loss = []
train_acc = []
test_loss = []
test_acc = []

for epoch in tqdm(range(epochs)):
    print(f"epoch: {epoch}")
    for batch, (X,y) in enumerate(train_data_loader):
      model_0.train()
      y_pred = model_0(X)
      loss = loss_fn(y_pred, y)
      acc = accuracy_fn(y, y_pred)
      train_acc.append(acc)
      train_loss.append(loss)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model_0.eval()
    with torch.inference_mode():
      for X_test, y_test in test_data_loader:
          test_pred = model_0(X_test)
          testloss = loss_fn(test_pred, y_test)
          testacc = accuracy_fn(y_test, test_pred)
          test_acc.append(testacc)
          test_loss.append(testloss)
    if batch % 1200 == 0:
        print(f"Number of samples {batch*len(X)}/{len(train_data_loader.dataset)}")
    print(f"Train_loss: {loss}| Test_loss: {testloss}|  Test_acc:{testacc}|Train_acc:{acc}")

In [ ]:
def eval_model(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn: nn.Module,
    accuracy_fn):
  loss, acc = 0,0
  model_0.eval()
  with torch.inference_mode():
    for X, y in tqdm(data_loader):
      y_pred = model_0(X)
      loss += loss_fn(y_pred, y)
      acc += accuracy_fn(y, y_pred)

    loss /= len(data_loader)
    acc /= len(data_loader)

  return {'model_name': model.__class__.__name__,
          'model_loss': loss.item(),
          'model_acc' : acc}

model_0_results = eval_model(model = model_0,
                             data_loader=test_data_loader,
                             loss_fn = loss_fn,
                             accuracy_fn= accuracy_fn)
model_0_results